In [50]:
from transliterate import translit, get_available_language_codes
import pymorphy2
import os
import re
import transliterate
import string
from collections import Counter
from nltk.corpus import stopwords

# Тося

## Скачивание файлов

In [2]:
folder = []
for i in os.walk('childes_Tosia'):
    folder.append(i)

Выводим список всех файлов и папок 

In [3]:
#folder

In [4]:
filenames = []
for address, dirs, files in folder:
    for file in files:
        filenames.append(file)

Выводим список всех файлов, содержащих диалоги

In [5]:
#filenames

In [6]:
paths = []  # список готовых директорий к файлам с диалогами
for filename in filenames:
    if filename.startswith('TD'):
        directory = 'childes_Tosia/malenkaya/'
    else:
        directory = 'childes_Tosia/'
    path = directory + filename
    paths.append(path)
    #print(path)

#print(len(paths))

## Разделение реплик на то, что ребенок слышит, и то, что он говорит

In [16]:
morph = pymorphy2.MorphAnalyzer()

dict_words = []

textChild = ''
textParents = ''

for filename in paths:
    with open(filename, 'r', encoding='utf-8') as f:
        file = f.readlines()
        for line in file:
            line = translit(line, 'ru')
            line = re.sub('йо', 'ё', line)
            if line.startswith('*ЦХИ'):  # выбираем только реплики ребенка 
                line = re.sub('"*".*', '', line)
                line = re.sub('[0-9_]', '', line)
                textChild += line.lower()
            elif line.startswith('*'):
                line = re.sub('"*".*', '', line)
                line = re.sub('[0-9_]', '', line)
                textParents += line.lower()
        
with open('TosiaSays.txt', 'w+', encoding='utf-8') as f:
    f.write(textChild)
with open('TosiaHears.txt', 'w+', encoding='utf-8') as f:
    f.write(textParents)

In [64]:
def WordsCounting(filepath):
    with open(filepath, 'r+', encoding='utf-8') as f:
        text = f.readlines()
        # список из всех слов, которые говорит или слышит ребенок
        words = []
        for line in text:
            for word in line.lower().translate(str.maketrans(dict.fromkeys(string.punctuation))).split():
                words.append(word)

    # список из словрей вида слово:часть речи
    dict_words = [] 
    for word in words:
        pos = morph.parse(word)[0].tag.POS
        d = {}
        d[word] = pos
        dict_words.append(d)
    
    # список из частей речи всех слов -- для частеречного анализа
    pos = []
    for d in dict_words:
        for values in d.values():
            if values != None:
                pos.append(values)
    
    # топ частей речи, которые слышит или говорит ребенок
    top_pos = Counter(pos)
    print('Распределение по частям речи ', top_pos)
    
    # топ-10 слов и топ-10 существительных, которые слышит или говорит ребенок
    all_words = []
    all_nouns = []
    forbidden = ['цхи', 'ххх', 'мот', 'фат', 'æто',
                 'лаугхинга', 'чико', 'нана', 'мотхер', 'бро',
                 'мота', 'xxx', 'сис']
    for d in dict_words:
        for key, value in d.items():
            key = morph.parse(key)[0].normal_form
            if key not in forbidden:
                all_words.append(key)
            if value == 'NOUN' and key not in forbidden:
                all_nouns.append(key)
                
    sw = stopwords.words('russian')
    
    filtered_words = [w for w in all_words if w not in sw]
    filtered_nouns = [w for w in all_nouns if w not in sw]
    
    top_words = Counter(filtered_words)
    top_words = dict(top_words)
    top_words = {c: top_words[c] for c in top_words if top_words[c] > 2 and len(c) > 2}
    top_words = (sorted(top_words.items(), key=lambda x: x[1], reverse=True))[:10]
    
    top_nouns = Counter(filtered_nouns)
    top_nouns = dict(top_nouns)
    top_nouns = {c: top_nouns[c] for c in top_nouns if top_nouns[c] > 2 and len(c) > 2}
    top_nouns = (sorted(top_nouns.items(), key=lambda x: x[1], reverse=True))[:10]
    
    print('Топ-10 всех слов', top_words)
    print('Топ-10 всех существительных', top_nouns)

### Что Тося говорит

In [65]:
WordsCounting('TosiaSays.txt')

Распределение по частям речи  Counter({'NOUN': 2692, 'NPRO': 1328, 'VERB': 1270, 'PRCL': 1237, 'CONJ': 559, 'ADVB': 525, 'ADJF': 480, 'PRED': 464, 'INTJ': 345, 'PREP': 295, 'INFN': 213, 'NUMR': 75, 'ADJS': 58, 'GRND': 45, 'COMP': 27, 'PRTS': 7})
Топ-10 всех слов [('xxx', 300), ('xочить', 102), ('няня', 100), ('ешчё', 88), ('ама', 83), ('смотреть', 77), ('видеть', 76), ('мочь', 62), ('большой', 62), ('варя', 54)]
Топ-10 всех существительных [('няня', 100), ('ешчё', 88), ('ама', 83), ('варя', 54), ('тося', 52), ('мама', 36), ('стасик', 33), ('шчас', 26), ('метр', 25), ('папа', 24)]


### Что Тося слышит

In [66]:
WordsCounting('TosiaHears.txt')

Распределение по частям речи  Counter({'NOUN': 16326, 'VERB': 4508, 'CONJ': 3016, 'NPRO': 2884, 'PRCL': 2788, 'ADVB': 1943, 'PREP': 1725, 'ADJF': 1632, 'INFN': 906, 'INTJ': 662, 'PRED': 431, 'ADJS': 405, 'GRND': 225, 'NUMR': 168, 'COMP': 93, 'PRTS': 39, 'PRTF': 16})
Топ-10 всех слов [('нанна', 567), ('тося', 458), ('сис', 302), ('давать', 301), ('смотреть', 163), ('ешчё', 161), ('xxx', 160), ('мочь', 139), ('сказать', 135), ('весь', 135)]
Топ-10 всех существительных [('нанна', 567), ('тося', 458), ('ешчё', 161), ('тосй', 109), ('мама', 102), ('шчас', 78), ('тоня', 67), ('варя', 63), ('свет', 59), ('молодец', 54)]


# Яша

## Скачивание файлов

In [13]:
folder = []
for i in os.walk('childes_Yasha'):
    folder.append(i)

Выводим список всех файлов и папок

In [26]:
#folder

In [21]:
filenames = []
for address, dirs, files in folder:
    for file in files:
        filenames.append(file)

Выводим список всех файлов, содержащих диалоги

In [19]:
#filenames

In [22]:
new = []
for filename in filenames:
    if filename[0].isnumeric():
        new.append(filename)
print(new)
print(len(new))

['10.txt', '11.txt', '12.txt', '13.06.2018.txt', '13.txt', '13_02_2018.txt', '14.txt', '15.txt', '16.txt', '18.txt', '19.txt', '20.txt', '21.cha', '22.cha', '22_02_2018.cha', '23.cha', '24.cha', '24_03_2018.cha', '25.cha', '26.cha', '27.01.2018.cha', '27.cha', '6_02_2018.txt', '7.txt', '8.txt']
25


In [25]:
paths = []  # список готовых директорий к файлам с диалогами
for filename in new:
    directory = 'childes_Yasha/'
    path = directory + filename
    paths.append(path)
    #print(path)

#print(len(paths))

## Разделение реплик на то, что ребенок слышит, и то, что он говорит

In [27]:
morph = pymorphy2.MorphAnalyzer()

dict_words = []

for filename in paths:
    with open(filename, 'r', encoding='utf-8') as f:
        file = f.readlines()
        textChild = ''
        textParents = ''
        for line in file:
            line = translit(line, 'ru')
            line = re.sub('йо', 'ё', line)
            if line.startswith('*ЦХИ'):  # выбираем только реплики ребенка 
                line = re.sub('"*".*', '', line)
                line = re.sub('[0-9_]', '', line)
                textChild += line.lower()
            elif line.startswith('*'):
                line = re.sub('"*".*', '', line)
                line = re.sub('[0-9_]', '', line)
                textParents += line.lower()
        
with open('YashaSays.txt', 'w+', encoding='utf-8') as f:
    f.write(textChild)
with open('YashaHears.txt', 'w+', encoding='utf-8') as f:
    f.write(textParents)

### Что Яша говорит

In [67]:
WordsCounting('YashaSays.txt')

Распределение по частям речи  Counter({'NOUN': 233, 'PREP': 53, 'INTJ': 33, 'VERB': 24, 'ADVB': 16, 'PRCL': 7, 'ADJF': 6, 'PRED': 5, 'NUMR': 4, 'ADJS': 3, 'CONJ': 3, 'INFN': 3})
Топ-10 всех слов [('ала', 18), ('картошка', 15), ('деу', 14), ('мемец', 12), ('мультик', 12), ('вилка', 10), ('дать', 10), ('тятёца', 9), ('пац', 7), ('алло', 7)]
Топ-10 всех существительных [('ала', 18), ('картошка', 15), ('мемец', 12), ('мультик', 12), ('вилка', 10), ('тятёца', 9), ('яша', 5), ('татёца', 5), ('папа', 4), ('миша', 4)]


### Что Яша слышит

In [68]:
WordsCounting('YashaHears.txt')

Распределение по частям речи  Counter({'NOUN': 1286, 'CONJ': 472, 'VERB': 452, 'NPRO': 300, 'PRCL': 277, 'ADVB': 155, 'PREP': 127, 'ADJF': 81, 'INFN': 63, 'INTJ': 60, 'PRED': 38, 'ADJS': 17, 'COMP': 8, 'NUMR': 6, 'GRND': 4, 'PRTS': 2, 'PRTF': 1})
Топ-10 всех слов [('яша', 109), ('написать', 37), ('говорить', 30), ('давать', 28), ('сказать', 24), ('спать', 24), ('ешчё', 20), ('пожалуйста', 19), ('картошка', 18), ('мусор', 17)]
Топ-10 всех существительных [('яша', 109), ('ешчё', 20), ('картошка', 18), ('мусор', 17), ('стол', 16), ('пупок', 16), ('салат', 12), ('буква', 12), ('коляска', 11), ('крокодил', 11)]
